# LSTM Regressor

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
sh = pd.read_csv('/content/drive/MyDrive/P-SAT/주제분석/모델링/shinhan_label(5일추가).csv')

In [ ]:
sh.columns

Index(['날짜', 'year', 'month', 'day', '종가', '대비', '등락률', '거래량', '거래대금', '시가총액',
       '외국인 보유수량', '외국인 지분율', '토론방', '순매수_기관', '순매수_기타법인', '순매수_개인', '순매수_외국인',
       '검색어', '보도량', '기사감성점수', '뉴스심리지수', '비트코인종가', '비트코인거래량', '비트코인변동',
       '코스피종가', '코스피대비', '코스피등락률', '코스피거래량', '코스피거래대금', '코스피시가총액', '한은금리',
       '원/미국달러', '원/위안', '원/일본엔(100엔)', '원/유로', '경제심리지수(원계열)', '경제심리지수(순환변동치)',
       '산업생산지수', '물가상승률', '소비자신뢰지수', '소비자심리지수', '경제활동참가율(%)', '실업률(%)',
       '고용률(%)', 'year-month', '5일 등락률', 'day5_label'],
      dtype='object')

In [ ]:
# 변수 선택

SH_df = sh[['경제심리지수(순환변동치)','시가총액','비트코인종가','원/미국달러','소비자심리지수','코스피거래대금','순매수_개인',
            '순매수_외국인','산업생산지수','코스피거래량','뉴스심리지수','원/유로','토론방','실업률(%)','원/일본엔(100엔)',
            '거래량','기사감성점수','외국인 보유수량','코스피등락률','경제활동참가율(%)','검색어','보도량','순매수_기관','비트코인거래량','비트코인변동', '5일 등락률', 'day5_label']]

In [ ]:
SH_df.index = sh['날짜'] # index 설정
SH_df=SH_df.dropna(axis=0) # 결측치 제거

## 라벨인코딩

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(SH_df['day5_label'].values)
SH_df['day5_label'] = le.transform(SH_df['day5_label'].values)

SH_df
# buy = 0 (매수)
# maintain = 1 (유지)
# sell = 2 (매도)

,경제심리지수(순환변동치),시가총액,비트코인종가,원/미국달러,소비자심리지수,코스피거래대금,순매수_개인,순매수_외국인,산업생산지수,코스피거래량,...,외국인 보유수량,코스피등락률,경제활동참가율(%),검색어,보도량,순매수_기관,비트코인거래량,비트코인변동,5일 등락률,day5_label
날짜,,,,,,,,,,,,,,,,,,,,,
2017-06-15,99.6,2.375740e+13,2442.5,1125.6,110.8,5882836.0,1.919981e+09,-6.272512e+08,103.9,334886.0,...,329144439.0,-0.46,63.9,24.17292,20.0,-1.157351e+09,262790.0,-1.01,1.42,1
2017-06-16,99.6,2.399450e+13,2508.6,1122.8,110.8,5965450.0,-7.252675e+09,-6.899162e+08,103.9,329179.0,...,329126090.0,0.01,63.9,21.66271,18.0,8.486123e+09,117210.0,2.71,1.40,1
2017-06-19,99.6,2.380482e+13,2616.8,1133.0,110.8,5421942.0,3.520761e+08,-3.957569e+09,103.9,290945.0,...,329112398.0,0.38,63.9,26.60542,17.0,3.620935e+09,81830.0,3.04,-0.20,1
2017-06-20,99.6,2.352030e+13,2754.4,1131.4,110.8,7143556.0,7.991526e+09,-4.312687e+09,103.9,360734.0,...,328995796.0,-0.07,63.9,25.71450,20.0,-3.723991e+09,111300.0,5.26,-1.00,1
2017-06-21,99.6,2.297497e+13,2677.6,1137.0,110.8,6466060.0,1.490063e+10,-1.026999e+10,103.9,360345.0,...,328908691.0,-0.49,63.9,24.37307,17.0,-9.242260e+09,116030.0,-2.79,-4.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,89.8,1.773115e+13,27129.8,1289.7,92.0,8534650.0,5.688245e+09,-9.705766e+09,112.5,472847.0,...,316673932.0,-0.24,64.1,21.76489,35.0,3.973928e+09,107240.0,-3.02,0.43,1
2023-03-28,89.8,1.829082e+13,27262.2,1300.3,92.0,8812618.0,-7.743957e+08,-8.388950e+09,112.5,524016.0,...,316650435.0,1.07,64.1,18.59415,38.0,9.395792e+09,94160.0,0.49,2.86,1
2023-03-29,89.8,1.841801e+13,28350.4,1297.4,92.0,8829889.0,1.748838e+09,-8.002469e+09,112.5,497218.0,...,316418844.0,0.37,64.1,16.96578,38.0,6.080323e+09,109320.0,3.99,2.12,1


In [ ]:
# Minmax scaling을 코드로 구현

def normalize_data(dataset): 
  # X, y 모두 정규화

  cols = dataset.columns.tolist()
  col_name = [0]*len(cols)

  for i in range(len(cols)):
    col_name[i]=i

  dataset.columns = col_name
  dtypes = dataset.dtypes.tolist()
  minmax = list()

  for column in dataset:
    dataset = dataset.astype({column: 'float32'})

  for i in range(len(cols)):
    col_values = dataset[col_name[i]]
    value_min = min(col_values)
    value_max = max(col_values)
    minmax.append([value_min,value_max])

  for column in dataset:
    values = dataset[column].values
    for i in range(len(values)):
      values[i]=(values[i]-minmax[column][0])/(minmax[column][1]-minmax[column][0])
    dataset[column] = values

  dataset[column]=values

  return dataset, minmax

In [ ]:
label = SH_df['day5_label'].values
dataset, minmax = normalize_data(SH_df) # minmax scaling 적용

dataset[dataset.shape[1]-1] = label 
values = dataset.values
dataset

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
날짜,,,,,,,,,,,,,,,,,,,,,
2017-06-15,0.787234,0.838486,0.003202,0.179420,0.968992,0.071275,0.566756,0.468977,0.272031,0.046066,...,0.941201,0.466745,0.750000,0.114595,0.250000,0.400307,0.000059,0.589589,0.519478,1
2017-06-16,0.787234,0.853724,0.004215,0.172032,0.968992,0.073266,0.507937,0.468540,0.272031,0.044321,...,0.940860,0.494408,0.750000,0.085284,0.222222,0.479377,0.000026,0.647050,0.519050,1
2017-06-19,0.787234,0.841534,0.005872,0.198945,0.968992,0.060172,0.556702,0.445745,0.272031,0.032633,...,0.940606,0.516186,0.750000,0.142998,0.208333,0.439486,0.000018,0.652147,0.484803,1
2017-06-20,0.787234,0.823249,0.007979,0.194723,0.968992,0.101647,0.605689,0.443268,0.272031,0.053967,...,0.938443,0.489700,0.750000,0.132595,0.250000,0.379263,0.000025,0.686438,0.467680,1
2017-06-21,0.787234,0.788204,0.006803,0.209499,0.968992,0.085326,0.649994,0.401711,0.272031,0.053848,...,0.936828,0.464979,0.750000,0.116932,0.208333,0.334017,0.000026,0.562095,0.394050,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,0.489362,0.451212,0.381294,0.612401,0.483204,0.135160,0.590920,0.405647,0.601532,0.088239,...,0.709876,0.479694,0.799999,0.086477,0.458333,0.442380,0.000024,0.558542,0.498288,1
2023-03-28,0.489362,0.487178,0.383322,0.640370,0.483204,0.141856,0.549479,0.414833,0.601532,0.103880,...,0.709441,0.556798,0.799999,0.049453,0.500000,0.486835,0.000021,0.612759,0.550300,1
2023-03-29,0.489362,0.495353,0.399988,0.632718,0.483204,0.142272,0.565659,0.417529,0.601532,0.095688,...,0.705145,0.515597,0.799999,0.030439,0.500000,0.459651,0.000024,0.666821,0.534461,1


In [ ]:
# windowing 함수 : 시계열 데이터의 학습 데이터셋과 테스트 데이터셋을 만들기 위해 제작

def split_sequences(sequence, n_steps): 

  X, y = list(), list()
  
  for i in range(len(sequence)):
    end_ix = i + n_steps

    if end_ix > len(sequence)-1:
      # 윈도우 사이즈 통일
      break

    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)

  return np.array(X), np.array(y)

In [ ]:
def data_setup(n_steps, sequence):

  X, y = split_sequences(sequence, n_steps) # windowing 내부적으로 진행
  n_features = X.shape[2] # 변수 개수
  X = X.reshape((len(X), n_steps, n_features))

  new_y = []
  for term in y:
    new_term = term[-1] # 예측할 변수
    new_y.append(new_term)

  return X, np.array(new_y), n_features

In [ ]:
n_steps = 10 # windowing 시점
X, y, n_features = data_setup(n_steps, values) # n_seq

In [ ]:
rel_val_len = 1
rel_test_len = 0.5 # test set 비율

# validation test split

X_test,y_test = X[:int(len(X)*rel_test_len)],y[:int(len(X)*rel_test_len)]
X_val,y_val = X[int(len(X)*rel_test_len):int(len(X)*rel_val_len)],y[int(len(X)*rel_test_len):int(len(X)*rel_val_len)]

print("X : {}, {}".format(X_val.shape, X_test.shape))

X : (709, 10, 27), (708, 10, 27)


## LSTM load

In [ ]:
CHECKPOINT_PATH = '/content/drive/MyDrive/P-SAT/주제분석/모델링/checkpoints/model.checkpoint12_8000.pt'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math
import random

In [ ]:
seed = 11
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [ ]:
# 텐서로 형변환

X_val = torch.Tensor(X_val)
X_test = torch.Tensor(X_test)
y_val = torch.Tensor(y_val)
y_test = torch.Tensor(y_test)

In [ ]:
# 차원 변환

y_val = y_val.view([y_val.shape[0],1])
y_test = y_test.view([y_test.shape[0],1])

In [ ]:
print(y_val.shape)
print(y_test.shape)

torch.Size([709, 1])
torch.Size([708, 1])


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable 

class LSTM(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTM,self).__init__()
    self.num_classes = num_classes
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.seq_length = seq_length
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, 
                        num_layers = num_layers, batch_first = True)
    self.fc_1 =  nn.Linear(hidden_size, 128) # fully connected 1
    self.fc_2 = nn.Linear(128, num_classes) #fully connected last layer
    self.relu = nn.ReLU()

  def forward(self, x):
    h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) # 0으로 초기화한 hidden state
    c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) # 0으로 초기화한 cell state
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) # lstm 계층에 hidden state, cell state 적용
    hn = hn.view(-1, self.hidden_size) # 은닉층 위해서 reshaping the data 
    out = self.relu(hn)
    out = self.fc_1(out)
    out = self.relu(out) 
    out = self.fc_2(out)
    return out


In [ ]:
# train할 때와 동일한 설정
num_classes = 1 # output class의 개수
input_size = 27 # X variable의 개수
hidden_size = 2  # 은닉층의 뉴런 개수
num_layers = 1 # LSTM 계층수
seq_length = 10

In [ ]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, seq_length)

In [ ]:
learning_rate = 0.0001

criterion = torch.nn.MSELoss() 
optimizer = optim.Adam(lstm.parameters(), lr = learning_rate)

In [ ]:
# 저장된 checkpoint 불러오기

checkpoint = torch.load(CHECKPOINT_PATH)
lstm.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

lstm.eval()

LSTM(
  (lstm): LSTM(27, 2, batch_first=True)
  (fc_1): Linear(in_features=2, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
# validation data 적용

pred_val = lstm(X_val) # 예측
loss_val = criterion(pred_val, y_val) # test set loss function 계산

print('Validation set loss : {}'.format(loss_val))

Validation set loss : 0.12993189692497253


In [ ]:
pred_val = pred_val.view([pred_val.shape[0]])
y_val = y_val.view([y_val.shape[0]])

pred_val = np.array(pred_val.tolist())
y_val = np.array(y_val.tolist())

## threshold 결정 함수 : 3개 ( 매수, 매도, 유지 정확도 ) 평균

In [ ]:
def threshold_catcher(cm):
  buy_acc = cm[0,0]/sum(cm[0]) # 매수 정확도
  sell_acc = cm[2,2]/sum(cm[2]) # 매도 정확도
  maintain_acc = cm[1,1]/sum(cm[1]) # 유지 정확도

  rst = np.mean([buy_acc, sell_acc, maintain_acc])
  return rst 

In [ ]:
pred_min_val = int(round(pred_val.min(), 1)*10)
pred_max_val = int(round(pred_val.max(), 1)*10)

In [ ]:
# validation set에서 가장 성능이 좋은 threshold 결정

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

pred_bi = pred_val.copy()
threshold = [x / 10.0 for x in range(pred_min_val, pred_max_val, 1)]

num = 0
best_threshold_value = 0
best_th1 = 0
best_th2 = 0

for i in threshold:
  for j in threshold:
    if i < j:
      pred_bi = pred_val.copy()
      for k in range(len(pred_bi)):
        if pred_bi[k] < i:
          pred_bi[k] = 0
        elif pred_bi[k] < j:
          pred_bi[k] = 1
        else:
          pred_bi[k] = 2 

      cm = confusion_matrix(y_val, pred_bi)
      if threshold_catcher(cm) > best_threshold_value:
        best_threshold_value = threshold_catcher(cm)
        best_th1 = i
        best_th2 = j

## TEST

In [ ]:
# test data 적용

pred_test = lstm(X_test) # 예측
loss_test = criterion(pred_test, y_test) # test set loss function 계산

print('Test set loss : {}'.format(loss_test))

Test set loss : 0.11772327125072479


In [ ]:
pred_test = pred_test.view([pred_test.shape[0]])
y_test = y_test.view([y_test.shape[0]])

pred_test = np.array(pred_test.tolist())
y_test = np.array(y_test.tolist())

In [ ]:
# SK하이닉스
# 앞의 과정(trial and error)을 통해 결정한 threshold 
lowerb = best_th1
upperb = best_th2

pred_bi = pred_test.copy()

for k in range(len(pred_bi)):
  if pred_bi[k] < lowerb:
    pred_bi[k] = 0
  elif pred_bi[k] < upperb:
    pred_bi[k] = 1
  else:
    pred_bi[k] = 2 

cm = confusion_matrix(y_test, pred_bi)
print('========== SK하이닉스 ==========')
print(cm)
print()
print('전체 정확도 : ', accuracy_score(y_test, pred_bi))
print('전체 f1-score : ', f1_score(y_test, pred_bi, average='weighted'))
print()
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ', cm[2,2]/sum(cm[2]))
print('유지 정확도 : ', cm[1,1]/sum(cm[1]))
print()
print('매수 정밀도 : ', cm[0,0]/np.sum(cm, axis=0)[0])
print('매도 정밀도 : ', cm[2,2]/np.sum(cm, axis=0)[2])

========== SK하이닉스 ==========
[[104  18   0]
 [113 313  45]
 [  3  27  76]]

전체 정확도 :  0.7052932761087267
전체 f1-score :  0.7165113879684445

매수 정확도 :  0.8524590163934426
매도 정확도 :  0.7169811320754716
유지 정확도 :  0.6645435244161358

매수 정밀도 :  0.4727272727272727
매도 정밀도 :  0.628099173553719


In [ ]:
# 신한지주
# 앞의 과정(trial and error)을 통해 결정한 threshold 
lowerb = best_th1
upperb = best_th2

pred_bi = pred_test.copy()

for k in range(len(pred_bi)):
  if pred_bi[k] < lowerb:
    pred_bi[k] = 0
  elif pred_bi[k] < upperb:
    pred_bi[k] = 1
  else:
    pred_bi[k] = 2 

cm = confusion_matrix(y_test, pred_bi)
print('========== 신한지주 ==========')
print(cm)
print()
print('전체 정확도 : ', accuracy_score(y_test, pred_bi))
print('전체 f1-score : ', f1_score(y_test, pred_bi, average='weighted'))
print()
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ', cm[2,2]/sum(cm[2]))
print('유지 정확도 : ', cm[1,1]/sum(cm[1]))
print()
print('매수 정밀도 : ', cm[0,0]/np.sum(cm, axis=0)[0])
print('매도 정밀도 : ', cm[2,2]/np.sum(cm, axis=0)[2])

========== 신한지주 ==========
[[ 23  15   0]
 [ 24 529  62]
 [  1  15  39]]

전체 정확도 :  0.8347457627118644
전체 f1-score :  0.8503657789754228

매수 정확도 :  0.6052631578947368
매도 정확도 :  0.7090909090909091
유지 정확도 :  0.8601626016260162

매수 정밀도 :  0.4791666666666667
매도 정밀도 :  0.38613861386138615


In [ ]:
# 현대차
# 앞의 과정(trial and error)을 통해 결정한 threshold 
lowerb = best_th1
upperb = best_th2

pred_bi = pred_test.copy()

for k in range(len(pred_bi)):
  if pred_bi[k] < lowerb:
    pred_bi[k] = 0
  elif pred_bi[k] < upperb:
    pred_bi[k] = 1
  else:
    pred_bi[k] = 2 

cm = confusion_matrix(y_test, pred_bi)
print('========== 현대차 ==========')
print(cm)
print()
print('전체 정확도 : ', accuracy_score(y_test, pred_bi))
print('전체 f1-score : ', f1_score(y_test, pred_bi, average='weighted'))
print()
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ', cm[2,2]/sum(cm[2]))
print('유지 정확도 : ', cm[1,1]/sum(cm[1]))
print()
print('매수 정밀도 : ', cm[0,0]/np.sum(cm, axis=0)[0])
print('매도 정밀도 : ', cm[2,2]/np.sum(cm, axis=0)[2])

========== 현대차 ==========
[[  9   9   3]
 [ 47 248  57]
 [  4   7  14]]

전체 정확도 :  0.6809045226130653
전체 f1-score :  0.7416229778038823

매수 정확도 :  0.42857142857142855
매도 정확도 :  0.56
유지 정확도 :  0.7045454545454546

매수 정밀도 :  0.15
매도 정밀도 :  0.1891891891891892
